<a href="https://colab.research.google.com/github/Benji8bit/NetoNUMPY_53/blob/main/neto_numpy_hw4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Задание 1**
> **Обязательная часть**
Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).
Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:
KEYWORDS = ['python', 'парсинг']
Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).
В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [ ]:
import sys
sys.path.append('../')
from library.habr.lib import HabrPage

habrPage = HabrPage.from_internet()

In [ ]:
import pandas as pd

keywords = ['powerpoint', 'бизнес']
posts = habrPage.find_by_keywords(keywords)

titles = [post.title for post in posts]
times = [post.time for post in posts]
urls = [post.url for post in posts]

pd.DataFrame({'заголовок': titles, 'дата': times, 'ссылка': urls})

# > Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.
Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.
Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

# **Задание 2**
> **Обязательная часть**
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]
В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>
Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

# Получаем данные из API

In [11]:
import requests
import json

EMAIL = ["xxx@x.ru", "yyy@y.com"]

url = "https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data"
data = {"emailAddresses": EMAIL}
headers = {
    "Vaar-Version": "0",
    "Vaar-Header-App-Product": "hackcheck-web-avast",
    "Vaar-Header-App-Product-Name": "hackcheck-web-avast",
    "Vaar-Header-App-Build-Version": "1.0.0",
}
r = requests.post(url=url, data=json.dumps(data), headers=headers)

# Формируем датафрейм по утечкам

In [ ]:
import pandas as pd

def generate_breaches_dataframe(avast_response):
    avast_response_json = avast_response.json()

    breaches = pd.DataFrame()
    for breach in avast_response_json['breaches'].values():
        # remove extra column
        del breach['statistics']
        # parse datetime
        breach['publishDate'] = pd.to_datetime(breach['publishDate'])
        # generate dataframe for one record
        df = pd.DataFrame(breach.values(), index=breach.keys())
        # transpose and accumulate it
        breaches = breaches.append(df.T)

    return breaches.reset_index(drop=True)

breaches = generate_breaches_dataframe(r)
breaches.head()

# Формируем датафрейм по ящикам

In [ ]:
def generate_emails_dataframe(avast_response):
    emails = pd.DataFrame()
    for email, breaches in r.json()['summary'].items():
        # prepare list of id
        breachIds = list(breaches.values())[0]
        # generate dataframe and transpose
        df = pd.DataFrame.from_dict({'email': [email], 'breachId': breachIds}, orient='index').T
        # for every breachId set current email
        df.email.fillna(email, inplace=True)
        # accumulate dataframes
        emails = emails.append(df)

    return emails.reset_index(drop=True)


emails = generate_emails_dataframe(r)
emails.head()

# Мержим датафреймы и выводим результат

In [ ]:
merged = emails.merge(breaches, on='breachId')
merged[['email', 'publishDate', 'site', 'description']]